<a href="https://colab.research.google.com/github/Akanksha-Thippani/SentimentAnalyse/blob/main/Copy_of_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade pip
!pip install groq pandas gradio python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from getpass import getpass
raw_key = getpass("Paste your GROQ API key (hidden): ")


Paste your GROQ API key (hidden): ··········


In [ ]:
import os, json, re
import pandas as pd

# Install groq if not already installed
try:
    from groq import Groq
except ImportError:
    !pip install groq
    from groq import Groq


# ✅ updated model name (llama3-70b-8192)
DEFAULT_MODEL = "openai/gpt-oss-20b"  # supported model


client = Groq(api_key=raw_key)

def analyze_transcript_with_groq(transcript: str, model: str = DEFAULT_MODEL):
    """
    Summarise transcript and classify sentiment using Groq's chat completion API.
    """
    system = {"role":"system",
              "content":"You summarise customer calls and classify sentiment."}
    user = {"role":"user", "content": (
        "Return ONLY this JSON:\n"
        '{"summary":"<2-3 sentence summary>", "sentiment":"Positive|Neutral|Negative|Mixed"}\n\n'
        "Transcript:\n" + transcript
    )}
    resp = client.chat.completions.create(messages=[system, user], model=model)
    raw_text = resp.choices[0].message.content

    # Try strict JSON parse first
    try:
        parsed = json.loads(raw_text)
        return parsed.get("summary","").strip(), parsed.get("sentiment","").strip(), raw_text
    except Exception:
        # fallback: find JSON-like substring
        m = re.search(r'\{.*\}', raw_text, re.S)
        if m:
            try:
                parsed = json.loads(m.group())
                return parsed.get("summary","").strip(), parsed.get("sentiment","").strip(), raw_text
            except Exception:
                pass
        # final fallback
        summary, sentiment = "", ""
        for line in raw_text.splitlines():
            if line.lower().startswith("summary"):
                summary = line.split(":",1)[1].strip()
            if line.lower().startswith("sentiment"):
                sentiment = line.split(":",1)[1].strip()
        if not summary: summary = raw_text[:250]
        if not sentiment: sentiment = "Unknown"
        return summary, sentiment, raw_text

def save_row_to_csv(transcript, summary, sentiment,
                    path="/content/drive/MyDrive/call_analysis.csv"):
    """
    Append results to a CSV file in Google Drive.
    """
    # Check if the directory exists before saving
    dir_path = os.path.dirname(path)
    if not os.path.exists(dir_path):
        raise OSError(f"Cannot save file into a non-existent directory: '{dir_path}'. Please ensure Google Drive is mounted correctly.")

    df = pd.DataFrame([{"Transcript": transcript,
                        "Summary": summary,
                        "Sentiment": sentiment}])
    if os.path.exists(path):
        df.to_csv(path, mode='a', header=False, index=False)
    else:
        df.to_csv(path, index=False)
    return path

In [ ]:
sample = """Agent: Hello, how can I help you?
Customer: I tried to book a slot yesterday but payment failed and I got charged twice."""

summary, sentiment, raw = analyze_transcript_with_groq(sample)
print("Summary:", summary)
print("Sentiment:", sentiment)
print("Raw model output:\n",raw)







Summary: The customer attempted to book a slot and encountered a payment failure, resulting in being charged twice. They are seeking resolution for the duplicate charge and the unsuccessful transaction.
Sentiment: Negative
Raw model output:
 {"summary":"The customer attempted to book a slot and encountered a payment failure, resulting in being charged twice. They are seeking resolution for the duplicate charge and the unsuccessful transaction.","sentiment":"Negative"}


In [ ]:
import gradio as gr

def process_and_save(transcript):
    summary, sentiment, raw = analyze_transcript_with_groq(transcript)
    saved = save_row_to_csv(transcript, summary, sentiment)
    return summary, sentiment, raw + "\n\nSaved to: " + saved

with gr.Blocks() as demo:
    gr.Markdown("## Call Transcript Analyzer")
    inp = gr.Textbox(label="Transcript", lines=8)
    out_sum = gr.Textbox(label="Summary", interactive=False)
    out_sent = gr.Textbox(label="Sentiment", interactive=False)
    out_raw = gr.Textbox(label="Raw model output", lines=6, interactive=False)
    btn = gr.Button("Analyze & Save")
    btn.click(fn=process_and_save, inputs=inp,
              outputs=[out_sum, out_sent, out_raw])

demo.launch(share=True, debug=True)



Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e125951ff142a5b329.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
